In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import pprint
import time
from datetime import datetime, timezone
import json
import random
from deep_translator import GoogleTranslator

Put the twitter accounts that you want to crawl in the 'niji_screen_name_list'

In [ ]:
niji_screen_name_list=['elonmusk','BarackObama','DonaldJTrumpJr']
print(niji_screen_name_list)
print(f'length of list:{len(niji_screen_name_list)}')

greater_screen_name_list=niji_screen_name_list
greater_uid_list=[]

How to get a header:
1. login your twitter bot account (better not be your main account)
2. open anybody's twitter detailed page
3. Press F12 (Chrome) to start developer tool
4. Search 'User' in the Network column of the tool
5. Refresh this page
6. Click on an item like 'TweetDetail'
7. Switch to 'Headers' column
8. Copy all the items in Request Headers listed in the next block


In [ ]:
#parameters for twitter uid


headers={
    'Accpet':'*/*',
    'accept-encoding':'gzip, deflate, br, zstd',
    'accept-language':'zh-CN,zh;q=0.9,ja;q=0.8,en;q=0.7,zh-TW;q=0.6,ru;q=0.5',
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    "authorization": "",
    'cookie':'',
    "x-csrf-token": ""
}

features = {
    "responsive_web_grok_bio_auto_translation_is_enabled": True,
    "hidden_profile_subscriptions_enabled": True,
    "payments_enabled": True,
    "profile_label_improvements_pcf_label_in_post_enabled": True,
    "rweb_tipjar_consumption_enabled": True,
    "verified_phone_label_enabled": True,
    "subscriptions_verification_info_is_identity_verified_enabled": True,
    "subscriptions_verification_info_verified_since_enabled": True,
    "highlights_tweets_tab_ui_enabled": True,
    "responsive_web_twitter_article_notes_tab_enabled": True,
    "subscriptions_feature_can_gift_premium": True,
    "creator_subscriptions_tweet_preview_api_enabled": True,
    "responsive_web_graphql_skip_user_profile_image_extensions_enabled": True,
    "responsive_web_graphql_timeline_navigation_enabled": True
}

fieldToggles = {
    "withAuxiliaryUserLabels": True
}




In [ ]:
import requests
import json

def get_user_id(screen_name, headers):
    url = "https://x.com/i/api/graphql/x3RLKWW1Tl7JgU7YtGxuzw/UserByScreenName"
    variables = {
        "screen_name": screen_name,
        "withSafetyModeUserFields": True,
        "withSuperFollowsUserFields": True
    }
    params = {
        "variables": json.dumps(variables),
        "features": json.dumps(features),
        "fieldToggles": json.dumps(fieldToggles)
    }
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=10)
        if resp.status_code == 404:
            print(f"用户不存在: {screen_name}")
            return None
        elif resp.status_code != 200:
            print(f"请求失败: 状态码 {resp.status_code} {screen_name}")
            return None
        data = resp.json()
        # 取用户ID
        user_id = data['data']['user']['result']['rest_id']
        return user_id
    except Exception as e:
        print(f"异常: {e}")
        return None


for i in range(len(greater_screen_name_list)):
  user_id=get_user_id(greater_screen_name_list[i],headers)
  print(user_id)
  greater_uid_list.append(user_id)
  time.sleep(5)

print('uid_list:')
print(greater_uid_list)
print('Sleep...600s')
time.sleep(600) #leave it sleep to avoid being blocked by twitter




In [ ]:
#parameters for tweets

variables = {
    "userId": "",
    "count": 20,
    "cursor": "",
    "includePromotedContent": True,  
    "withQuickPromoteEligibilityTweetFields": True,
    "withVoice": True
}
features = {
    "verified_phone_label_enabled": False,
    "responsive_web_grok_image_annotation_enabled": True,
    "c9s_tweet_anatomy_moderator_badge_enabled": True,
    "articles_preview_enabled": True,
    "graphql_is_translatable_rweb_tweet_is_translatable_enabled": True,
    "tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled": True,
    "responsive_web_edit_tweet_api_enabled": True,
    "creator_subscriptions_quote_tweet_preview_enabled": False,
    "responsive_web_grok_analyze_button_fetch_trends_enabled": True,
    "responsive_web_grok_analysis_button_from_backend": True,
    "communities_web_enable_tweet_community_results_fetch": True,
    "responsive_web_twitter_article_tweet_consumption_enabled": True,
    "responsive_web_graphql_skip_user_profile_image_extensions_enabled": False,
    "responsive_web_grok_community_note_auto_translation_is_enabled": False,
    "payments_enabled": False,
    "longform_notetweets_inline_media_enabled": True,
    "responsive_web_enhance_cards_enabled": True,
    "responsive_web_grok_show_grok_translated_post": False,
    "responsive_web_jetfuel_frame": True,
    "creator_subscriptions_tweet_preview_api_enabled": True,
    "rweb_tipjar_consumption_enabled": True,
    "tweet_awards_web_tipping_enabled": False,
    "premium_content_api_read_enabled": False,
    "responsive_web_grok_share_attachment_enabled": True,
    "longform_notetweets_consumption_enabled": True,
    "responsive_web_grok_analyze_post_followups_enabled": True,
    "standardized_nudges_misinfo": True,
    "responsive_web_graphql_timeline_navigation_enabled": True,
    "freedom_of_speech_not_reach_fetch_enabled": True,
    "profile_label_improvements_pcf_label_in_post_enabled": True,
    "rweb_video_screen_enabled": False,
    "longform_notetweets_rich_text_read_enabled": True,
    "view_counts_everywhere_api_enabled": True
}
params = {
    "variables": json.dumps(variables),
    "features": json.dumps(features),
    "fieldToggles": json.dumps({"withArticlePlainText": False})
}



In [ ]:
#url = "https://api.twitter.com/1.1/application/rate_limit_status.json"
#resp = requests.get(url, headers=headers)
#print(resp.json())

The next block is the core code.
It will visit the target's homepage timeline first, and then visit each twitter on its timeline, then each reply in the twitter.
If other targets appaear in the reply, they will be recorded.

In [ ]:
cursor=""
sender_list=[]
receiver_list=[]
text_list=[]
time_list=[]
apiurl = "https://x.com/i/api/graphql/MdhoJlJzYWas9JNmFz7H3A/UserTweets"
startindex=0 #could change starting from which one

for i in range(startindex,len(greater_uid_list)):#every person
    print(f'i:{i}')
    cursor=""
    uid=str(greater_uid_list[i])
    for k in range(2):#every person's timeline page, number of pages can be changed here

        variables = {
        "userId": f"{uid}",
        "count": 20,
        "cursor": cursor,
        "includePromotedContent": False,  
        "withQuickPromoteEligibilityTweetFields": True,
        "withVoice": True
        }
        params = {
        "variables": json.dumps(variables),
        "features": json.dumps(features),
        "fieldToggles": json.dumps({"withArticlePlainText": False})
        }
        resp=requests.get(headers=headers,url=apiurl,params=params)#request for timeline page
        print(f"[{greater_screen_name_list[i]}] 主页{k}页状态码: {resp.status_code}")
        #print(resp.text[:500])

        resp_json = resp.json()
        #print(resp_json)
        instructions = resp_json['data']['user']['result']['timeline']['timeline']['instructions']

        entries = None
        for ins in instructions:
            if 'entries' in ins:
                entries = ins['entries']
                break

        if not entries:
            print("⚠ 没有找到 entries，可能 timeline 为空或结构变了")
            continue  # 跳过这个用户

        #find the index of entries in instructions
        resp_json_instruction=resp_json['data']['user']['result']['timeline']['timeline']['instructions']
        for m in range(len(resp_json_instruction)):
            if "entries" in instructions[m]:
                print(f'entries index:{m}')
                resp_json_entries=resp_json_instruction[m]['entries']#comfirm entries(tweets) list
        


        #print(resp_json_entries)
        for item in resp_json_entries:#every tweets in every person's timeline page
            if not item:
                continue
            tweet_id = item['entryId'].replace("tweet-", "", 1)
            if not tweet_id.isdigit():
                print(f"跳过非数字tweet_id: {tweet_id}")
                continue
            print(f'已提取tweetid:{tweet_id}')

            variables = {
                "focalTweetId": tweet_id,
                "with_rux_injections": False,
                "includePromotedContent": False,
                "withCommunity": True,
                "withQuickPromoteEligibilityTweetFields": True,
                "withBirdwatchNotes": True,
                "withVoice": True
            }


            features = {
                "verified_phone_label_enabled": False,
                "responsive_web_grok_image_annotation_enabled": True,
                "c9s_tweet_anatomy_moderator_badge_enabled": True,
                "articles_preview_enabled": True,
                "graphql_is_translatable_rweb_tweet_is_translatable_enabled": True,
                "tweet_with_visibility_results_prefer_gql_limited_actions_policy_enabled": True,
                "responsive_web_edit_tweet_api_enabled": True,
                "creator_subscriptions_quote_tweet_preview_enabled": False,
                "responsive_web_grok_analyze_button_fetch_trends_enabled": True,
                "responsive_web_grok_analysis_button_from_backend": True,
                "communities_web_enable_tweet_community_results_fetch": True,
                "responsive_web_twitter_article_tweet_consumption_enabled": True,
                "responsive_web_graphql_skip_user_profile_image_extensions_enabled": False,
                "responsive_web_grok_community_note_auto_translation_is_enabled": False,
                "payments_enabled": False,
                "longform_notetweets_inline_media_enabled": True,
                "responsive_web_enhance_cards_enabled": True,
                "responsive_web_grok_show_grok_translated_post": False,
                "responsive_web_jetfuel_frame": True,
                "creator_subscriptions_tweet_preview_api_enabled": True,
                "rweb_tipjar_consumption_enabled": True,
                "tweet_awards_web_tipping_enabled": False,
                "premium_content_api_read_enabled": False,
                "responsive_web_grok_share_attachment_enabled": True,
                "longform_notetweets_consumption_enabled": True,
                "responsive_web_grok_analyze_post_followups_enabled": True,
                "standardized_nudges_misinfo": True,
                "responsive_web_graphql_timeline_navigation_enabled": True,
                "freedom_of_speech_not_reach_fetch_enabled": True,
                "profile_label_improvements_pcf_label_in_post_enabled": True,
                "rweb_video_screen_enabled": False,
                "longform_notetweets_rich_text_read_enabled": True,
                "view_counts_everywhere_api_enabled": True
            }


            field_toggles = {
            "withArticlePlainText": False
            }

            params = {
            "variables": json.dumps(variables),
            "features": json.dumps(features),
            "fieldToggles": json.dumps(field_toggles)
            }

            url = "https://x.com/i/api/graphql/FJGOFKfjA67MmT4I9p1qZg/TweetDetail"

            resp_tweet = requests.get(url, headers=headers, params=params)#requests for detailed tweet
            print(f'{greater_screen_name_list[i]}具体推特响应码:{resp_tweet.status_code}')
            data = resp_tweet.json().get('data', {})

            threaded = data.get('threaded_conversation_with_injections_v2')

            if threaded and 'instructions' in threaded:
                instructions = threaded['instructions']
            else:
                print("⚠ 该推文没有 'threaded_conversation_with_injections_v2' 字段，可能没有评论或者接口结构变了。")
                print(resp_tweet.json())  # 可以打印看看具体返回什么
                #跳过或做其他处理
                instructions = []

            # 找到带有 entries 的 instruction
            entries = []
            for ins in instructions:
                if ins.get('entries'):
                    entries = ins['entries']
                    break

            if not entries:
                print("⚠ 没有找到 entries，可能这条推文没有评论。")
                continue

            length=len(entries)# limited to 10 replies per tweet
            if length >10:
                length = 10

            try:
                tweet_create_year = str(entries[0]['content']['itemContent']['tweet_results']['result']['legacy']['created_at'])[-4:]
                if tweet_create_year != '2025':
                    print('End of result in 2025')
                    break
            except KeyError:
                print("缺少 legacy 字段，跳过")
    

            for j in range(1, length):#every comment in every detailed tweet
                entry = entries[j]
                entry_id=str(entries[j]['entryId'])
                if 'conversationthread' not in entry_id:
                    #no sign of comment
                    continue
                items = entry.get('content', {}).get('items')
                if not items:
                    # 没有items，跳过这个entry
                    continue
                for ia in range(len(items)):
                    tweet_result = items[ia]['item']['itemContent'].get('tweet_results')
                    if not tweet_result:
                        print("⚠ 无 tweet_results，跳过该条 item")
                        continue
                    user_result= items[ia]['item']['itemContent']['tweet_results']['result']
                    typename = user_result.get("__typename")
                    if typename != 'Tweet':
                        print(f"⚠️ 跳过非 Tweet 类型: {typename}")
                        continue
                    user_info = user_result['core']['user_results']['result']
                    try:
                        responder_name = user_info['core']['screen_name']
                        respond_text = user_result['legacy']['full_text']
                        respond_time = user_result['legacy']['created_at']
                    except KeyError as e:
                        print(f"⚠️ 缺少字段 {e}，跳过该条 item")
                        continue
                    respond_receiver = items[ia]['item']['itemContent']['tweet_results']['result']['legacy'].get('in_reply_to_screen_name')
                    if respond_receiver is None:
                        respond_receiver=greater_screen_name_list[i]
                    if (
                        (responder_name in greater_screen_name_list) 
                        and (respond_receiver in greater_screen_name_list)
                        and (respond_receiver != responder_name)
                    ):
                        text_list.append(respond_text)
                        time_list.append(respond_time)
                        sender_list.append(responder_name)
                        receiver_list.append(respond_receiver)
                        print(f'{responder_name}->{respond_receiver}:{respond_text}, at {respond_time}')
                        lottery_num=random.randint(1,10)
                        if lottery_num == 10:
                            print(f'中奖了！休眠100秒')
                            time.sleep(100)
                time.sleep(random.uniform(0,1))
            time.sleep(random.uniform(5,10))#every tweet

        cursor=resp_json_entries[-1]['content']['value']
        print(f'cursor:{cursor}')
        time.sleep(random.uniform(5, 7))#every page
        gmt_time = datetime.now(timezone.utc)
        print("GMT Time:", gmt_time.strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(random.uniform(60, 100))#every user
    if i % 10 == 0 and i != 0:
        long_rest=random.uniform(1000,2000)
        print(f'Sleep {long_rest}s')
        time.sleep(long_rest)

print('已完成搜索！')


#The sleeping time can change depending on your needs, but remember never going too fast

In [ ]:
reply_df = pd.DataFrame({
    'sender': sender_list,
    'receiver': receiver_list,
    'text_content': text_list,
    'time stamp': time_list
})
print(reply_df)

In [ ]:
#cleaned df without unrelated repliers
reply_df_2 = reply_df[
    (reply_df['receiver'].isin(greater_screen_name_list)) & (reply_df['receiver'] != reply_df['sender'])
].copy()


In [ ]:
#Sample Data
#import pandas as pd
#reply_df_2 = pd.DataFrame({
#    'sender': ['Alice', 'Bob', 'Alice', 'Charlie', 'Bob', 'Charlie', 'Alice', 'Dave'],
#    'receiver': ['Bob', 'Alice', 'Charlie', 'Alice', 'Dave', 'Bob', 'Dave', 'Alice'],
#    'text_content': [
#        'Hi Bob, how are you?',
#        'Hey Alice! Doing well.',
#        'Hi Charlie, long time no see.',
#        'Hello Alice! Nice to hear from you.',
#        'Hey Dave, are you coming?',
#        'Hi Bob, I am on my way.',
#        'Hello Dave! Good luck today.',
#        'Thanks Alice, see you!'
#    ]
#})

The next block is about cleaning the data and translation, you could skip it if you want.

In [ ]:
#Data Cleaning
#Transalation and vader


from deep_translator import GoogleTranslator
from time import sleep
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

translator = GoogleTranslator(source='auto', target='en')
sid_obj = SentimentIntensityAnalyzer()


def remove_emoji(text):
    import re
    emoji_pattern = re.compile(
    "["u"\U0001F600-\U0001F64F"
    u"\U0001F300-\U0001F5FF"
    u"\U0001F680-\U0001F6FF"
    u"\U0001F1E0-\U0001F1FF"
    u"\U00002700-\U000027BF"
    u"\U0001F900-\U0001F9FF"
    u"\U0001FA70-\U0001FAFF"
    u"\U00002600-\U000026FF"
    u"\u200d\u2640-\u2642\u2600-\u2B55\ufe0f]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def safe_translate(text):
    if not text.strip():
        return ""
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"翻译失败: {e} -> {text}")
        return ""

def rate_limited_translate(text, i):
    if i % 10 == 0:
        sleep(1)
    return safe_translate(text)

reply_df_2['cleaned_text'] = reply_df_2['text_content'].apply(remove_emoji)
reply_df_2['translated_text_content'] = [
    rate_limited_translate(txt, idx) for idx, txt in enumerate(reply_df_2['cleaned_text'])
]
reply_df_2['vader_score'] = reply_df_2['translated_text_content'].apply(lambda x: abs(sid_obj.polarity_scores(x)['compound']))



Finally, generating a Multidigraph in the form of html.

In [ ]:
from pyvis.network import Network
import networkx as nx

# 创建 MultiDiGraph
G = nx.MultiDiGraph()

# 添加边：sender → receiver
for i, row in reply_df_2.iterrows():
    sender = row['sender']
    receiver = row['receiver']
    weight = row['vader_score']
    text = str(row['text_content'])

    # 边 key 用评论前10字（可调整）
    key = text[:10] if len(text) > 10 else text
    G.add_edge(sender, receiver, key=key, weight=weight, text=text)

# 创建 PyVis 网络
net = Network(height="800px", width="100%", notebook=False, directed=True)

# 添加节点
for node in G.nodes:
    net.add_node(node, label=node, title=f"用户: {node}", color="skyblue", size=20)

# 添加边
for u, v, k, data in G.edges(keys=True, data=True):
    text = data.get('text', '(无内容)')
    weight = data.get('weight', 0)
    color = "blue" if weight > 0 else "red"
    tooltip = f"评论: {text}<br>情感分数: {weight:.2f}"
    net.add_edge(u, v, title=tooltip, value=abs(weight), color=color)

# 生成 HTML 文件
save_path = r"D:\python-for-excel\web crawler\tweet_crawler\greater_twitter_network_test.html"# change your path here
net.save_graph(save_path)

net.show("graph.html", notebook=False)
from IPython.display import IFrame
IFrame("graph.html", width=800, height=600)



The following blocks calculate centrality based on unweighted network.

In [ ]:
import networkx as nx
G = nx.MultiDiGraph()


degree_list=[]
# 添加边：sender → receiver
for i, row in reply_df_2.iterrows():
  sender = row['sender']
  receiver = row['receiver']
  text = str(row['text_content'])

  degree=(sender,receiver)
  if degree in degree_list:
     continue
  else:
    degree_list.append(degree)

  # 边 key 用评论前10字（可调整）
  key = text[:10] if len(text) > 10 else text
  G.add_edge(sender, receiver, key=key,text=text)

In [ ]:
import pandas as pd
import networkx as nx

# 计算中心性指标
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

# 整合到 DataFrame
centrality_df = pd.DataFrame({
    'node': list(degree_centrality.keys()),
    'degree_centrality': list(degree_centrality.values()),
    'closeness_centrality': list(closeness_centrality.values()),
    'betweenness_centrality': list(betweenness_centrality.values()),
})

# 按 degree_centrality 降序排列
centrality_df.sort_values('degree_centrality', ascending=False, inplace=True)
print(centrality_df)